# Cross-Session Memory for Strands Agents

This notebook demonstrates how to implement cross-session memory for Strands Agents. We'll simulate multiple sessions and show how an agent can recall information from previous interactions.

## 1. Setup Memory Manager

In [ ]:
import json
import os
from typing import Dict, List, Any, Optional
from datetime import datetime
from strands.agent.conversation_manager import ConversationManager
from strands.types.content import Message, Messages

class CrossSessionMemoryManager(ConversationManager):
    """A conversation manager that maintains memory across sessions."""
    
    def __init__(self, memory_id: str, storage_dir: str = "memory_storage", max_sessions: int = 10):
        """
        Initialize the cross-session memory manager.
        
        Args:
            memory_id: Unique identifier for this memory instance
            storage_dir: Directory where memory files will be stored
            max_sessions: Maximum number of session histories to retain
        """

        super().__init__()
        self.memory_id = memory_id
        self.storage_dir = storage_dir
        self.max_sessions = max_sessions
        self.memory_file = os.path.join(storage_dir, f"{memory_id}.json")
        
        # Current session state
        self.session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        self._messages = []
        
        # All sessions memory
        self.sessions = {}
        self.summaries = {}
        
        # Ensure storage directory exists
        os.makedirs(storage_dir, exist_ok=True)
        
        # Load existing memory if available
        self._load_memory()
    
    def _load_memory(self):
        """Load memory from storage file if it exists."""
        if os.path.exists(self.memory_file):
            try:
                with open(self.memory_file, 'r') as f:
                    memory_data = json.load(f)
                    self.sessions = memory_data.get('sessions', {})
                    self.summaries = memory_data.get('summaries', {})
                    print(f"Loaded memory with {len(self.sessions)} previous sessions")
            except Exception as e:
                print(f"Error loading memory from {self.memory_file}: {e}")
                self.sessions = {}
                self.summaries = {}
    
    def _save_memory(self):
        """Save memory to storage file."""
        try:
            # Update session history
            self.sessions[self.session_id] = self._messages
            
            # Keep only the most recent sessions
            if len(self.sessions) > self.max_sessions:
                # Sort sessions by date (assuming ID format includes sortable date)
                sorted_sessions = sorted(self.sessions.keys())
                # Remove oldest sessions
                for old_session in sorted_sessions[:-self.max_sessions]:
                    del self.sessions[old_session]
            
            # Save to file
            memory_data = {
                'sessions': self.sessions,
                'summaries': self.summaries
            }
            with open(self.memory_file, 'w') as f:
                json.dump(memory_data, f, indent=2)
        except Exception as e:
            print(f"Error saving memory to {self.memory_file}: {e}")
    
    def add_message(self, message: Message) -> None:
        """Add a message to the current session and save to memory."""
        self._messages.append(message)
        self._save_memory()
    
    def get_messages(self) -> List[Message]:
        """Get messages for the current session."""
        return self._messages
    
    def add_session_summary(self, summary: str) -> None:
        """Add a summary for the current session."""
        self.summaries[self.session_id] = summary
        self._save_memory()
    
    def get_session_history(self, include_summaries: bool = True) -> str:
        """Get formatted history of past sessions."""
        if not self.sessions:
            return "No previous sessions found."
        
        history = []
        for session_id in sorted(self.sessions.keys()):
            if session_id == self.session_id:  # Skip current session
                continue
                
            # Format date from session ID
            try:
                date_part = session_id.split('_')[0]
                formatted_date = f"{date_part[:4]}-{date_part[4:6]}-{date_part[6:]}"
            except:
                formatted_date = session_id
            
            history.append(f"Session from {formatted_date}:")
            
            # Add summary if available
            if include_summaries and session_id in self.summaries:
                history.append(f"Summary: {self.summaries[session_id]}\n")
            
            # Add conversation snippets - just first and last exchanges
            session_messages = self.sessions[session_id]
            if session_messages:
                # First exchange
                for idx, msg in enumerate(session_messages[:2]):
                    role = "User" if msg["role"] == "user" else "Assistant"
                    content = msg["content"]
                    # Truncate long messages
                    if len(content) > 100:
                        content = content[:97] + "..."
                    history.append(f"{role}: {content}")
                
                if len(session_messages) > 3:
                    history.append("...")
                
                # Last exchange (if different from first)
                if len(session_messages) > 2:
                    for msg in session_messages[-2:]:
                        role = "User" if msg["role"] == "user" else "Assistant"
                        content = msg["content"]
                        # Truncate long messages
                        if len(content) > 100:
                            content = content[:97] + "..."
                        history.append(f"{role}: {content}")
            
            history.append("")
        
        return "\n".join(history)
    
    def start_new_session(self) -> None:
        """Start a new session while preserving memory."""
        # Save current session
        self._save_memory()
        
        # Create new session
        self.session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        self._messages = []
        print(f"Started new session with ID: {self.session_id}")
    
    def clear(self) -> None:
        """Clear all memory."""
        self._messages = []
        self.sessions = {}
        self.summaries = {}
        
        if os.path.exists(self.memory_file):
            os.remove(self.memory_file)
            print(f"Cleared all memory and removed {self.memory_file}")

## 2. Set Up Agent with Memory

In [12]:
from strands import Agent
from strands.models import BedrockModel
from strands_tools import calculator, current_time
from strands.tools import tool

# Initialize memory manager
memory_manager = CrossSessionMemoryManager(memory_id="user_assistant_memory")

# Add a tool to access previous session history
@tool
def recall_previous_sessions() -> str:
    """Recall information from previous sessions with the user."""
    return memory_manager.get_session_history()

# Set up model
bedrock_model = BedrockModel(model_id="us.amazon.nova-pro-v1:0")

# System prompt that emphasizes memory usage
system_prompt = """
You are a helpful AI assistant with memory of past interactions with the user.
When appropriate, refer to information you've learned about the user in previous sessions.
If you don't have relevant memories or they seem outdated, you can ask for clarification.
You can use the recall_previous_sessions tool to access memories from past conversations.
"""

# Create agent with memory
memory_agent = Agent(
    model=bedrock_model,
    tools=[calculator, current_time, recall_previous_sessions],
    system_prompt=system_prompt,
    conversation_manager=memory_manager
)

TypeError: Can't instantiate abstract class CrossSessionMemoryManager without an implementation for abstract methods 'apply_management', 'reduce_context'

## 3. Session 1: Initial Interaction

In [ ]:
# First interaction to establish some personal context
response = memory_agent("Hello! My name is Alex and I'm a software developer working primarily with Python and JavaScript.")
print(f"Agent: {response.message}")

In [ ]:
# Follow-up to add more personal context
response = memory_agent("I'm currently working on a machine learning project for image classification. I've been struggling with dataset preparation.")
print(f"Agent: {response.message}")

In [ ]:
# Add a session summary
memory_manager.add_session_summary("Initial introduction. User is Alex, a software developer working with Python and JavaScript. Currently working on an image classification ML project with dataset preparation challenges.")

## 4. Simulate Session 2: Next Day

In [ ]:
# Start a new session to simulate a different conversation time
memory_manager.start_new_session()

In [ ]:
# Ask a follow-up question that references previous context
response = memory_agent("I've made progress on my project. The dataset is now properly labeled, but I'm having issues with model performance.")
print(f"Agent: {response.message}")

In [ ]:
# Add new information
response = memory_agent("I'm thinking of switching from a CNN to a vision transformer architecture. What do you think?")
print(f"Agent: {response.message}")

In [ ]:
# Add a session summary
memory_manager.add_session_summary("Follow-up on ML project. User has fixed dataset labeling issues but is having model performance problems. Considering switching from CNN to vision transformer.")

## 5. Simulate Session 3: A Week Later

In [ ]:
# Start another new session
memory_manager.start_new_session()

In [ ]:
# Open-ended greeting to test memory recall
response = memory_agent("Hello again! I hope you're doing well today.")
print(f"Agent: {response.message}")

In [ ]:
# Ask about project status, expecting the agent to remember context
response = memory_agent("I'm starting a new project now. This one involves natural language processing for customer support automation.")
print(f"Agent: {response.message}")

## 6. Analyze Memory Usage

In [ ]:
# Check session history
print(memory_manager.get_session_history())

In [ ]:
# View memory file content
import json

with open(memory_manager.memory_file, 'r') as f:
    memory_data = json.load(f)
    print(f"Memory file size: {os.path.getsize(memory_manager.memory_file) / 1024:.2f} KB")
    print(f"Number of sessions: {len(memory_data['sessions'])}")
    print(f"Session summaries: {json.dumps(memory_data['summaries'], indent=2)}")

## 7. Memory Management Tools

In [ ]:
@tool
def summarize_current_session() -> str:
    """Generate and save a summary of the current conversation session."""
    # In a real implementation, this would use the model to generate a summary
    # For demo purposes, we'll create a simple summary
    
    messages = memory_manager.get_messages()
    user_messages = [msg["content"] for msg in messages if msg["role"] == "user"]
    
    if not user_messages:
        return "No messages to summarize."
        
    # Create a simple summary from user messages
    summary = f"Conversation about {', '.join(user_messages[:2])[:100]}..."
    memory_manager.add_session_summary(summary)
    
    return f"Session summarized as: {summary}"

# Add the tool to the agent
memory_agent.tools = [calculator, current_time, recall_previous_sessions, summarize_current_session]

In [ ]:
# Test the summary tool
response = memory_agent("Can you summarize our current conversation?")
print(f"Agent: {response.message}")

## 8. Memory Search for Enhanced Recall

In [ ]:
@tool
def search_memory(query: str) -> str:
    """Search through memory for relevant information based on a query."""
    # In a production system, this would use embeddings/semantic search
    # For this demo, we'll use simple keyword matching
    
    results = []
    for session_id, messages in memory_manager.sessions.items():
        session_matches = []
        
        # Format date from session ID
        try:
            date_part = session_id.split('_')[0]
            formatted_date = f"{date_part[:4]}-{date_part[4:6]}-{date_part[6:]}"
        except:
            formatted_date = session_id
            
        # Search through messages in this session
        for msg in messages:
            content = msg.get('content', '').lower()
            if query.lower() in content:
                role = "User" if msg["role"] == "user" else "Assistant"
                # Highlight matching part
                highlighted = content.replace(
                    query.lower(), 
                    f"[{query.lower()}]"
                )
                session_matches.append(f"{role}: {highlighted}")
        
        if session_matches:
            results.append(f"\nFrom session {formatted_date}:")
            results.extend(session_matches)
    
    if not results:
        return f"No memories found matching '{query}'."
        
    return "\n".join([f"Search results for '{query}':"]+results)

# Add the search tool to the agent
memory_agent.tools = [
    calculator, current_time, recall_previous_sessions, 
    summarize_current_session, search_memory
]

In [ ]:
# Test the memory search
response = memory_agent("What do you remember about my machine learning project?")
print(f"Agent: {response.message}")

## 9. Clearing Memory (Optional)

For testing or privacy reasons, you may want to clear memory. Uncomment to run.

In [ ]:
# Uncomment to clear all memory
# memory_manager.clear()

## 10. Conclusion

This notebook demonstrated how to implement persistent, cross-session memory for Strands Agents. Key components include:

1. **Memory persistence** - Storing conversations across sessions
2. **Session management** - Creating and managing multiple conversation sessions
3. **Memory recall** - Providing tools for the agent to access previous information
4. **Session summaries** - Condensing key information for efficient retrieval
5. **Memory search** - Finding specific information across all stored sessions

These capabilities enable agents to provide more personalized and contextually relevant responses by building on previous interactions, rather than starting from scratch in each conversation.